# Table of Contents
* [Wrangle](#Wrangle)
* [Monthly Data](#Monthly-Data)
* [Consistently Viewed](#Consistently-Viewed)
  * [DS](#Data-Science(ds)-Conistently-Viewed)
  * [WebDev](#WebDev-Conistently-Viewed)
* [Cohorts That View Lessons Significantly](#Cohorts-Referring-to-Lessons-More-than-Other-Cohorts)
  * [WebDev](#WebDev-Cohort-Lessons-Views)
  * [DS](#DS-Cohort-Lessons-Views)
* [Least Accessed Lessons](#Lest-Accessed-Lessons-(web-pages))
  * [WebDev](#WebDev-Lest-Accessed-Lessons)
  * [DS](#DS-Lest-Accessed-Lessons)
* [Student Hardly Viewing Lessons](#Students-Hardly-Viewing-Lessons)
  * [WebDev](#WebDev-Least-Active-Users)
  * [DS](#DS-Least-Active-Users)
* [Pages Accessed After Graduation](#Pages-Accessed-After-Graduation)
  * [By Program](#By-Program)
* [Supicious Activity](#Suspicious-Activity)
  * [Possible Web Scraper](#Possible-Web-Scraper!)

In [1]:
import numpy as np
import pandas as pd
import math
from sklearn import metrics

from scipy.stats import entropy

import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates #to format dates on our plots
%matplotlib inline
import seaborn as sns
import wrangle

# This is to make sure matplotlib doesn't throw the following error:
# The next line fixes "TypeError: float() argument must be a string or a number, not 'Timestamp' matplotlib"
pd.plotting.register_matplotlib_converters()

print("Initalized")

Initalized


# Wrangle
[Table Of Content](#Table-of-Contents)

In [2]:
help(wrangle)

Help on module wrangle:

NAME
    wrangle

FUNCTIONS
    student_data()
        Files for this ask were provided off network and connect be retrived from the web. You must request access to them from the DBA
        
        * Pulls in the CodeUp anonymized student access data from a text file into a DF and renames the columns .
        * Merges the cohort data from the cohort.csv file with the student data
        * Sets the index to datetime
        * Creates a classtype, program, and location columns from program_id...
        and knowing which cohort belongs to which program ID
        * Creates seperate data frames for webdev and data science (ds) users
        * returns data, webdev, ds, and cohort dataframes (must grab all in this order)

FILE
    c:\users\anthony\documents\data_science\codeup\anomaly-project\wrangle.py




In [3]:
# pull in the dataframes from the wrangle function
data, web, ds, cohort = wrangle.student_data()
# print the heads of the data frames to ensure the wrangle function worked properly
data.head(1)

,date,time,page_viewed,user_id,cohort_id,ip,hour,day,month,name,start_date,end_date,program_id,classtype,program,location
datetime,,,,,,,,,,,,,,,,
2018-01-26 09:55:03,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,9,Friday,January,Hampton,2015-09-22,2016-02-06,1.0,Web-SanAntonio,Web,SanAntonio


In [4]:
web.head(1)

,date,time,page_viewed,user_id,cohort_id,ip,hour,day,month,name,start_date,end_date,program_id,classtype,program,location
datetime,,,,,,,,,,,,,,,,
2018-01-26 09:55:03,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,9,Friday,January,Hampton,2015-09-22,2016-02-06,1.0,Web-SanAntonio,Web,SanAntonio


In [5]:
ds.head(1)

,date,time,page_viewed,user_id,cohort_id,ip,hour,day,month,name,start_date,end_date,program_id,classtype,program,location
datetime,,,,,,,,,,,,,,,,
2019-08-20 09:39:58,2019-08-20,09:39:58,/,466,34.0,97.105.19.58,9,Tuesday,August,Bayes,2019-08-19,2020-01-30,3.0,DataScience-SanAntonio,DataScience,SanAntonio


In [6]:
cohort.head(1)

,cohort_id,name,start_date,end_date,program_id
0,1,Arches,2014-02-04,2014-04-22,1


# Monthly Data
To make the data processing easier we will resample the data by month and get the size (number of hits) for each cohort and page. We will use the ds_month and web_month in much of our analysis.


[Table Of Content](#Table-of-Contents)

In [7]:
# resample the index by month and set reset the index to the 'datetime' column 
ds_month = ds.groupby(['name', 'page_viewed']).resample('M').size().reset_index().set_index('datetime')
web_month = web.groupby(['name', 'page_viewed']).resample('M').size().reset_index().set_index('datetime')
# Rename the colums to name (cohort name), page (page_viewed), and hits (.size() result)
ds_month.columns=['name', 'page', 'hits']
web_month.columns=['name', 'page', 'hits']

Print each newly created data frame to ensure we get the results we expect

In [8]:
web_month

,name,page,hits
datetime,,,
2019-03-31,Andromeda,/,132
2019-04-30,Andromeda,/,232
2019-05-31,Andromeda,/,188
2019-06-30,Andromeda,/,110
2019-07-31,Andromeda,/,108
...,...,...,...
2019-07-31,Zion,uploads/598dc43df39e2.jpg,1
2019-12-31,Zion,web-design/intro,1
2019-07-31,Zion,web-design/ui/visuals,1


In [9]:
ds_month

,name,page,hits
datetime,,,
2019-08-31,Bayes,/,255
2019-09-30,Bayes,/,304
2019-10-31,Bayes,/,326
2019-11-30,Bayes,/,228
2019-12-31,Bayes,/,211
...,...,...,...
2020-10-31,Darden,timeseries/working-with-time-series-data,75
2020-11-30,Darden,timeseries/working-with-time-series-data,8
2020-10-31,Darden,timeseries/working-with-time-series-data-with-...,2


# Consistently Viewed
> Which lesson appears to attract the most traffic consistently across cohorts (per program)?


To answer this question, I will take the ds and webdev programs and determine the total number of times each page was viewed by each cohort. After determining the number of times each cohort viewed each webpage I will get the total of views of the web page across the cohorts and then take the average. 

[Table Of Content](#Table-of-Contents)

### Data Science(ds) Conistently Viewed

In [10]:
# group by cohort name and page and get the total number of hits per page per cohort
# Unstanck the dataframe to make the pages columns and reset column index to level one to access the page names directly
ds_sumhits = ds_month.groupby(['name', 'page']).sum().unstack().reset_index(col_level=1)
# Drop the top level column index as it's not needed
ds_sumhits.columns = ds_sumhits.columns.droplevel(0)
# reset the index
ds_sumhits = ds_sumhits.reset_index()
# transpose the results
ds_sumhits = ds_sumhits.transpose()
# drop the index and name rows as they are now extra
ds_sumhits = ds_sumhits.drop(labels=['index', 'name'])
# reset the index again and then set the index to page and drop the extra column created from this action
ds_sumhits = ds_sumhits.reset_index().set_index('page', drop=True)
# Grab the names of the cohorts from the ds_month dataframe as ds_columns
ds_columns=ds_month.name.unique()
# Assign the columns in the dataframe to the ds_columns
ds_sumhits.columns=ds_columns
# Fill all na values with 0
ds_sumhits = ds_sumhits.fillna(0)
# calculate the total number of hits across all cohorts
ds_sumhits['total_hits'] = ds_sumhits.sum(axis=1)
# Determine the average across all cohorts
ds_sumhits['total_mean'] = ds_sumhits.mean(axis=1)
# Print the newly created data frame where the average is above 300 views.
ds_sumhits[ds_sumhits['total_mean'] >= 300]

,Bayes,Curie,Darden,total_hits,total_mean
page,,,,,
/,1842.0,1523.0,2041.0,5406.0,2703.0
1-fundamentals/1.1-intro-to-data-science,615.0,461.0,460.0,1536.0,768.0
1-fundamentals/1.2-data-science-pipeline,221.0,146.0,271.0,638.0,319.0
1-fundamentals/AI-ML-DL-timeline.jpg,624.0,465.0,470.0,1559.0,779.5
1-fundamentals/DataToAction_v2.jpg,230.0,148.0,284.0,662.0,331.0
1-fundamentals/modern-data-scientist.jpg,626.0,467.0,470.0,1563.0,781.5
10-anomaly-detection/1-overview,383.0,345.0,14.0,742.0,371.0
10-anomaly-detection/AnomalyDetectionCartoon.jpeg,386.0,345.0,14.0,745.0,372.5
3-sql/1-mysql-overview,142.0,441.0,121.0,704.0,352.0


In [68]:
ds_sumhits[ds_sumhits['total_mean'] >= 300].to_csv('DS_Consistent_Viewed.csv')

### WebDev Conistently Viewed
[Table Of Content](#Table-of-Contents)

In [12]:
# group by cohort name and page and get the total number of hits per page per cohort
# Unstanck the dataframe to make the pages columns and reset column index to level one to access the page names directly
web_sumhits = web_month.groupby(['name', 'page']).sum().unstack().reset_index(col_level=1)
# Drop the top level column index as it's not needed
web_sumhits.columns = web_sumhits.columns.droplevel(0)
# reset the index
web_sumhits = web_sumhits.reset_index()
# transpose the results
web_sumhits = web_sumhits.transpose()
# drop the index and name rows as they are now extra
web_sumhits = web_sumhits.drop(labels=['index', 'name'])
# reset the index again and then set the index to page and drop the extra column created from this action
web_sumhits = web_sumhits.reset_index().set_index('page', drop=True)
# Grab the names of the cohorts from the web_month dataframe as web_columns
web_columns=web_month.name.unique()
# Assign the columns in the dataframe to the web_columns
web_sumhits.columns=web_columns
# Fill all na values with 0
web_sumhits = web_sumhits.fillna(0)
# calculate the total number of hits across all cohorts 
web_sumhits['total_hits'] = web_sumhits.sum(axis=1)
# calculate the average total_hits across the cohorts
web_sumhits['total_mean'] = web_sumhits.mean(axis=1)
# Print the newly created data frame where the average is above 300 views.
web_sumhits[web_sumhits['total_mean'] >= 300]

,Andromeda,Apex,Apollo,Arches,Badlands,Bash,Betelgeuse,Ceres,Deimos,Denali,...,Staff,Teddy,Ulysses,Voyageurs,Wrangell,Xanadu,Yosemite,Zion,total_hits,total_mean
page,,,,,,,,,,,,,,,,,,,,,
/,1156.0,1244.0,1.0,622.0,17.0,532.0,868.0,1620.0,1319.0,1.0,...,4633.0,1754.0,1618.0,2098.0,1112.0,916.0,962.0,1756.0,30668.0,1614.105263
appendix,161.0,426.0,0.0,133.0,0.0,80.0,180.0,408.0,287.0,0.0,...,989.0,488.0,174.0,393.0,209.0,320.0,142.0,289.0,6678.0,351.473684
html-css,372.0,708.0,0.0,215.0,2.0,214.0,425.0,753.0,609.0,0.0,...,977.0,225.0,555.0,528.0,405.0,587.0,362.0,649.0,10363.0,545.421053
java-i,336.0,528.0,0.0,132.0,1.0,157.0,422.0,510.0,434.0,0.0,...,624.0,501.0,375.0,639.0,401.0,483.0,245.0,593.0,8699.0,457.842105
java-ii,395.0,595.0,0.0,167.0,0.0,190.0,444.0,667.0,506.0,0.0,...,884.0,494.0,423.0,754.0,323.0,501.0,268.0,613.0,9956.0,524.000000
java-iii,430.0,616.0,0.0,153.0,0.0,72.0,444.0,674.0,517.0,0.0,...,908.0,712.0,411.0,770.0,335.0,495.0,340.0,742.0,10610.0,558.421053
javascript-i,509.0,610.0,0.0,294.0,1.0,319.0,686.0,977.0,696.0,0.0,...,1263.0,589.0,523.0,884.0,553.0,718.0,437.0,868.0,13902.0,731.684211
javascript-ii,290.0,431.0,0.0,204.0,0.0,175.0,374.0,519.0,392.0,0.0,...,744.0,468.0,308.0,584.0,278.0,493.0,332.0,637.0,8367.0,440.368421
jquery,335.0,437.0,0.0,133.0,0.0,233.0,540.0,632.0,466.0,0.0,...,615.0,498.0,363.0,583.0,374.0,543.0,278.0,559.0,8846.0,465.578947


In [67]:
web_sumhits[web_sumhits['total_mean'] >= 300].to_csv('Web_Consistent_Viewed.csv')

# Cohorts Referring to Lessons More than Other Cohorts
> Is there a cohort that referred to a lesson significantly more that other cohorts seemed to gloss over? 

To answer this question, we will use the web_hits and ds_hits DFs we created above to calculate the percentage of total hits of each cohort produced.

[Table Of Contents](#Table-of-Contents)

In [14]:
# Create a copy of the web_hits data frame called used
used = pd.DataFrame.copy(web_sumhits)
# using the web_columns object we created above, use a for loop to conduct turn each column into a percentage...
# of the total hits of each web page
for col in web_columns:
    used[col] = used[col]/used['total_hits']
# Drop the total_hits and total_mean columns
used = used.drop(columns=['total_hits', 'total_mean'])
# Transpose the data frame to make the cohorts the rows and the pages the columns 
used = used.transpose()
# drop the 'Staff' row
used = used.drop(labels='Staff', axis=0)
# Print the results
used

page,.git,.gitignore,.well-known/assetlinks.json,/,00_,00_index,01_intro,02_listing_files,03_file_paths,04_navigating_the_filesystem,...,web-design/ui/typography,web-design/ui/visuals,web-design/ux,web-design/ux/layout,web-design/ux/layout/.json,web-design/ux/purpose,web-dev-day-two,working-with-time-series-data,wp-admin,wp-login
Andromeda,0.0,0.0,0.0,0.037694,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.034749,0.050459,0.0,0.026316,0.0,0.037915,0.0,0.0,0.0,0.0
Apex,0.0,0.0,0.0,0.040563,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.077220,0.087156,0.0,0.074561,0.0,0.104265,0.0,0.0,0.0,0.0
Apollo,0.0,0.0,0.0,0.000033,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
Arches,0.0,0.0,0.0,0.020282,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
Badlands,0.0,0.0,0.0,0.000554,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
Bash,0.0,0.0,0.0,0.017347,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.004587,0.0,0.004386,0.0,0.000000,1.0,0.0,0.0,0.0
Betelgeuse,0.0,0.0,0.0,0.028303,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.100386,0.137615,0.0,0.166667,0.0,0.151659,0.0,0.0,0.0,0.0
Ceres,0.0,0.0,0.0,0.052824,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.127413,0.155963,0.0,0.149123,1.0,0.132701,0.0,0.0,0.0,0.0
Deimos,0.0,0.0,0.0,0.043009,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.057915,0.073394,0.0,0.065789,0.0,0.052133,0.0,0.0,0.0,0.0
Denali,0.0,0.0,0.0,0.000033,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [15]:
# grab the pages from the used data frame
pages = used.columns
# create an empty dataframe that uses the index from the used DF as the index only
web_top_views = pd.DataFrame(index=used.index)
# use the pages object to create a for loop that loops through each colummn and merges the web_top_views data frame with...
# the cohort-page pairs that have over 10% (.1) of the total views of that page across the program
# using a left join (to keep null values) and the indexes of each DF as the merge keys
for page in pages:
    web_top_views = web_top_views.merge(right=(pd.DataFrame(used[page][used[page]>=.1])), how='left', left_index=True, right_index=True)
# Concat the total_hits of each web page from the web_sumhits DF
web_top_views = pd.concat([web_top_views, web_sumhits[['total_hits']].transpose()])
# Print the results
web_top_views


page,.git,.gitignore,.well-known/assetlinks.json,/,00_,00_index,01_intro,02_listing_files,03_file_paths,04_navigating_the_filesystem,...,web-design/ui/typography,web-design/ui/visuals,web-design/ux,web-design/ux/layout,web-design/ux/layout/.json,web-design/ux/purpose,web-dev-day-two,working-with-time-series-data,wp-admin,wp-login
Andromeda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.104265,NaN,NaN,NaN,NaN
Apollo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Badlands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
Betelgeuse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.100386,0.137615,NaN,0.166667,NaN,0.151659,NaN,NaN,NaN,NaN
Ceres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.127413,0.155963,NaN,0.149123,1.0,0.132701,NaN,NaN,NaN,NaN
Deimos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Denali,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## WebDev Cohort Lessons Views
[Table Of Contents](#Table-of-Contents)

In [16]:
# Tranpose the DF to make the cohorts and the total hits rows the columns and the pages into the rows
web_top_views = web_top_views.transpose()
# eliminate any page that had under 500 total page views as we were told to find cohorts with significant traffic
web_top_views = web_top_views[web_top_views['total_hits']>= 500]
# retranspose the the DF so we can work on the df from a cohort perspective 
# This is necessary to make a mask that uses page values rather cohort values
web_top_views = web_top_views.transpose()
# create a mask where the average number of null values is lower than the max number of null values for page values
mask = web_top_views.isnull().mean() < ((len(web_top_views.index)-1)/(len(web_top_views.index)))
# retranspose the DF to then apply the mask to the DF on page values
web_top_views = web_top_views.transpose()
web_top_views = web_top_views[mask]
# create a row_mask that eliminates any row that has all null values
row_mask = web_top_views.isnull().mean(axis=0) < 1
# Tranpose the dataframe AGAIN to apply the mask to cohorts
web_top_views = web_top_views.transpose()
web_top_views = web_top_views[row_mask]
# Print the results for significantly more web view traffic for pages from different cohorts
web_top_views


page,appendix/capstone-workbook/sw-project-planning,appendix/documentation,appendix/extra-challenges/basic-challenges,appendix/further-reading/spring/authorization,appendix/further-reading/spring/devtools-configuration,appendix/professional-development/t-block-resume,appendix/slides,examples/css/messages.html,examples/javascript/ajax-store.html,html-css/css-i,...,javascript-i/testing-user-functions,jquery/simple-simon,mkdocs/search_index.json,search/search_index.json,slides/arrays,spring/extra-features/form-validation,spring/extra-features/json-response,spring/fundamentals/form-model-binding,spring/fundamentals/integration-tests,toc
Andromeda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.102908,0.105159,NaN,NaN,NaN
Apex,0.113798,NaN,NaN,NaN,NaN,0.182094,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.110912,NaN,NaN,NaN,NaN,0.175749,NaN
Bash,NaN,NaN,0.14032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Betelgeuse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ceres,0.146515,NaN,NaN,NaN,NaN,NaN,NaN,0.104668,NaN,NaN,...,0.108328,NaN,NaN,0.112134,NaN,NaN,NaN,NaN,0.123978,NaN
Deimos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.117166,NaN
Europa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fortuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.231660,NaN,NaN,NaN,0.122616,NaN
Ganymede,NaN,NaN,NaN,NaN,NaN,0.224583,0.118644,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Hyperion,NaN,NaN,NaN,NaN,NaN,0.209408,NaN,NaN,NaN,0.10095,...,0.101340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
web_top_views.to_csv('Web_Cohort_Pages.csv')

In [18]:
used = pd.DataFrame.copy(ds_sumhits)
for col in ds_columns:
    used[col] = used[col]/used['total_hits']
used = used.drop(columns=['total_hits', 'total_mean'])
used = used.transpose()
used

page,%20https://github.com/RaulCPena,",%20https://github.com/RaulCPena",/,1-fundamentals,1-fundamentals/1.1-intro-to-data-science,1-fundamentals/1.2-data-science-pipeline,1-fundamentals/1.3-pipeline-demo,1-fundamentals/2.1-excel-overview,1-fundamentals/2.1-spreadsheets-overview,1-fundamentals/2.2-excel-functions,...,timeseries/modeling-lesson2,timeseries/overview,timeseries/prep,timeseries/project,timeseries/sarimax,timeseries/svr,timeseries/working-with-time-series-data,timeseries/working-with-time-series-data-with-pandas,toc,working-with-time-series-data
Bayes,0.0,0.0,0.340733,0.0,0.400391,0.346395,0.433333,0.459893,0.043011,0.711628,...,0.063830,0.012987,0.048780,0.000000,0.142857,0.125,0.023256,0.25,1.0,0.0
Curie,1.0,1.0,0.281724,0.0,0.300130,0.228840,0.200000,0.245989,0.784946,0.125581,...,0.234043,0.103896,0.195122,0.014286,0.285714,0.125,0.011628,0.25,0.0,0.0
Darden,0.0,0.0,0.377543,1.0,0.299479,0.424765,0.366667,0.294118,0.172043,0.162791,...,0.702128,0.883117,0.756098,0.985714,0.571429,0.750,0.965116,0.50,0.0,1.0


In [19]:
pages = used.columns
ds_top_views = pd.DataFrame(index=used.index)
for page in pages:
    ds_top_views = ds_top_views.merge(right=(pd.DataFrame(used[page][used[page]>=.1])), how='left', left_index=True, right_index=True)
ds_top_views = pd.concat([ds_top_views, ds_sumhits[['total_hits']].transpose()])
ds_top_views


page,%20https://github.com/RaulCPena,",%20https://github.com/RaulCPena",/,1-fundamentals,1-fundamentals/1.1-intro-to-data-science,1-fundamentals/1.2-data-science-pipeline,1-fundamentals/1.3-pipeline-demo,1-fundamentals/2.1-excel-overview,1-fundamentals/2.1-spreadsheets-overview,1-fundamentals/2.2-excel-functions,...,timeseries/modeling-lesson2,timeseries/overview,timeseries/prep,timeseries/project,timeseries/sarimax,timeseries/svr,timeseries/working-with-time-series-data,timeseries/working-with-time-series-data-with-pandas,toc,working-with-time-series-data
Bayes,NaN,NaN,0.340733,NaN,0.400391,0.346395,0.433333,0.459893,NaN,0.711628,...,NaN,NaN,NaN,NaN,0.142857,0.125,NaN,0.25,1.0,NaN
Curie,1.0,1.0,0.281724,NaN,0.300130,0.228840,0.200000,0.245989,0.784946,0.125581,...,0.234043,0.103896,0.195122,NaN,0.285714,0.125,NaN,0.25,NaN,NaN
Darden,NaN,NaN,0.377543,1.0,0.299479,0.424765,0.366667,0.294118,0.172043,0.162791,...,0.702128,0.883117,0.756098,0.985714,0.571429,0.750,0.965116,0.50,NaN,1.0
total_hits,1.0,1.0,5406.000000,10.0,1536.000000,638.000000,300.000000,187.000000,93.000000,215.000000,...,47.000000,77.000000,123.000000,70.000000,7.000000,8.000,86.000000,8.00,9.0,14.0


## DS Cohort Lessons Views
Repeat the same process above, but for data science cohorts

[Table Of Contents](#Table-of-Contents)

In [20]:
# Tranpose the DF to make the cohorts and the total hits rows the columns and the pages into the rows
ds_top_views = ds_top_views.transpose()
# eliminate any page that had under 500 total page views as we were told to find cohorts with significant traffic
ds_top_views = ds_top_views[ds_top_views['total_hits']>= 500]
# retranspose the the DF so we can work on the df from a cohort perspective 
# This is necessary to make a mask that uses page values rather cohort values
ds_top_views = ds_top_views.transpose()
# create a mask where the average number of null values is lower than the max number of null values for page values
mask = ds_top_views.isnull().mean() < ((len(ds_top_views.index)-1)/(len(ds_top_views.index)))
# retranspose the DF to then apply the mask to the DF on page values
ds_top_views = ds_top_views.transpose()
ds_top_views = ds_top_views[mask]
# create a row_mask that eliminates any row that has all null values
row_mask = ds_top_views.isnull().mean(axis=0) < 1
# Tranpose the dataframe AGAIN to apply the mask to cohorts
ds_top_views = ds_top_views.transpose()
ds_top_views = ds_top_views[row_mask]
# Print the results for significantly more web view traffic for pages from different cohorts
ds_top_views


page,/,1-fundamentals/1.1-intro-to-data-science,1-fundamentals/1.2-data-science-pipeline,1-fundamentals/AI-ML-DL-timeline.jpg,1-fundamentals/DataToAction_v2.jpg,1-fundamentals/modern-data-scientist.jpg,10-anomaly-detection/1-overview,10-anomaly-detection/AnomalyDetectionCartoon.jpeg,3-sql/1-mysql-overview,5-stats/3-probability-distributions,5-stats/4.2-compare-means,6-regression/1-overview,appendix/cli-git-overview,classification/overview,classification/scale_features_or_not.svg,search/search_index.json,sql/mysql-overview
Bayes,0.340733,0.400391,0.346395,0.400257,0.347432,0.400512,0.516173,0.518121,0.201705,0.609524,0.568592,0.463936,0.537133,NaN,NaN,0.413974,NaN
Curie,0.281724,0.300130,0.228840,0.298268,0.223565,0.298784,0.464960,0.463087,0.626420,0.331429,0.355596,0.529831,0.328152,0.105814,NaN,0.360631,0.160976
Darden,0.377543,0.299479,0.424765,0.301475,0.429003,0.300704,NaN,NaN,0.171875,NaN,NaN,NaN,0.134715,0.882558,0.919003,0.225394,0.834146
total_hits,5406.000000,1536.000000,638.000000,1559.000000,662.000000,1563.000000,742.000000,745.000000,704.000000,525.000000,554.000000,1123.000000,579.000000,860.000000,642.000000,1331.000000,615.000000


In [21]:
ds_top_views.to_csv('DS_Cohort_Pages.csv')

# Lest Accessed Lessons (web-pages)
> Which lessons are least accessed?

To answer this question we get the total number of page views (hits) by each program and pull out the pages who's views were below the second quartile 

[Table Of Contents](#Table-of-Contents)

In [22]:
# create a data frame from the web program that pulls the total number of page views (hits)
web_lesson = pd.DataFrame(web.groupby(['page_viewed']).size())
# rename the column to hits to be consistent
web_lesson.columns=['hits']
# describe the data frame to get the quartiles
web_lesson.describe()

,hits
count,1939.000000
mean,312.842702
std,1343.568915
min,1.000000
25%,2.000000
50%,8.000000
75%,35.000000
max,30668.000000


## WebDev Lest Accessed Lessons
[Table Of Contents](#Table-of-Contents)

In [23]:
web_lesson[web_lesson['hits']<8]

,hits
page_viewed,
.git,1
.gitignore,1
.well-known/assetlinks.json,2
00_,1
06_moving_files,7
...,...
web-design/ux/layout/.json,1
web-dev-day-two,2
working-with-time-series-data,1


In [24]:
web_lesson.to_csv('Web_Least_Accessed.csv')

In [25]:
# create a data frame from the web program that pulls the total number of page views (hits)
ds_lesson = pd.DataFrame(ds.groupby(['page_viewed']).size())
# rename the column to hits to be consistent
ds_lesson.columns=['hits']
# describe the data frame to get the quartiles
ds_lesson.describe()

,hits
count,596.000000
mean,114.120805
std,282.876447
min,1.000000
25%,3.000000
50%,20.000000
75%,160.000000
max,5406.000000


## DS Lest Accessed Lessons
[Table Of Contents](#Table-of-Contents)

In [26]:
ds_lesson[ds_lesson['hits']<20]

,hits
page_viewed,
%20https://github.com/RaulCPena,1
",%20https://github.com/RaulCPena",1
1-fundamentals,10
1-fundamentals/2.3-visualization,4
1-fundamentals/2.3-visualization.md,5
...,...
timeseries/sarimax,7
timeseries/svr,8
timeseries/working-with-time-series-data-with-pandas,8


In [27]:
ds_lesson.to_csv('DS_Least_Accessed.csv')

# Students Hardly Viewing Lessons
> Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students

To answer this question we will create a WebDev and DS DFs that holds all the data of each user (students) and cohorts only when the cohort were in session. From there we will determine what users barely accessed the lessons based on daily total views. We use the whole data set because it's easier to split a single data frame rather than make 2 seperate ones.

[Table Of Contents](#Table-of-Contents)

In [28]:
# Create an empty data frame to store the data
active_class = pd.DataFrame()
# use zip within in a for loop to pull out the end date and cohort id combos from the cohort data frame
for cohort_id, end_date in zip(cohort.cohort_id, cohort.end_date):
    # Then for each cohort and end_date combo, only pull the rows where the class was still in session based on date
    active_class = active_class.append(data[(data['cohort_id'] == cohort_id) & (data['date'] <= end_date)])

active_class

,date,time,page_viewed,user_id,cohort_id,ip,hour,day,month,name,start_date,end_date,program_id,classtype,program,location
datetime,,,,,,,,,,,,,,,,
2018-01-26 10:08:22,2018-01-26,10:08:22,mkdocs/search_index.json,10,21.0,97.105.19.61,10,Friday,January,Sequoia,2017-09-27,2018-02-15,2.0,Web-Dallas,Web,Dallas
2018-01-26 10:08:34,2018-01-26,10:08:34,git/merge-conflict-demo,10,21.0,97.105.19.61,10,Friday,January,Sequoia,2017-09-27,2018-02-15,2.0,Web-Dallas,Web,Dallas
2018-01-26 10:08:45,2018-01-26,10:08:45,mkdocs/search_index.json,10,21.0,97.105.19.61,10,Friday,January,Sequoia,2017-09-27,2018-02-15,2.0,Web-Dallas,Web,Dallas
2018-01-26 10:08:55,2018-01-26,10:08:55,git/working-in-teams,10,21.0,97.105.19.61,10,Friday,January,Sequoia,2017-09-27,2018-02-15,2.0,Web-Dallas,Web,Dallas
2018-01-26 10:18:22,2018-01-26,10:18:22,/,10,21.0,97.105.19.61,10,Friday,January,Sequoia,2017-09-27,2018-02-15,2.0,Web-Dallas,Web,Dallas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-02 16:48:13,2020-11-02,16:48:13,javascript-i/coffee-project,763,62.0,107.192.148.199,16,Monday,November,Jupiter,2020-09-21,2021-03-30,2.0,Web-Dallas,Web,Dallas
2020-11-02 16:48:17,2020-11-02,16:48:17,javascript-i/mapbox-api,771,62.0,172.125.226.175,16,Monday,November,Jupiter,2020-09-21,2021-03-30,2.0,Web-Dallas,Web,Dallas
2020-11-02 16:48:18,2020-11-02,16:48:18,javascript-i/coffee-project,771,62.0,172.125.226.175,16,Monday,November,Jupiter,2020-09-21,2021-03-30,2.0,Web-Dallas,Web,Dallas


In [29]:
# turn the active_class to a grouped by user_id and name, 
# then resampled by day to get the number of hits each student-cohort pair
active_class = pd.DataFrame(active_class[['user_id', 'name', 'program']].groupby(['user_id', 'name', 'program']).resample('D').size())
# rename the column to hits
active_class.columns=['hits']
# rest the index to datetime
active_class = active_class.reset_index().set_index('datetime')
active_class

,user_id,name,program,hits
datetime,,,,
2018-01-26,2,Teddy,Web,17
2018-01-27,2,Teddy,Web,0
2018-01-28,2,Teddy,Web,0
2018-01-29,2,Teddy,Web,7
2018-01-30,2,Teddy,Web,51
...,...,...,...,...
2020-10-29,785,Darden,DataScience,2
2020-10-30,785,Darden,DataScience,4
2020-10-31,785,Darden,DataScience,0


In [30]:
# Average daily hits per user
users = active_class[['hits','user_id', 'name', 'program']].groupby(['user_id', 'name', 'program']).mean().reset_index().set_index('name')
users = users.rename(columns={'hits':'avg_daily_hits'})
users

,user_id,program,avg_daily_hits
name,,,
Teddy,2,Web,14.676190
Teddy,3,Web,13.889908
Teddy,4,Web,11.344262
Teddy,5,Web,15.292453
Teddy,6,Web,11.723214
...,...,...,...
Darden,780,DataScience,4.333333
Darden,781,DataScience,8.545455
Darden,783,DataScience,6.000000


In [31]:
# Average daily hits per cohort
active_cohort = active_class[['hits','name']].groupby('name').mean()
active_cohort.columns=['cohort_avg_daily_hits']
active_cohort

,cohort_avg_daily_hits
name,
Andromeda,7.520655
Apex,11.312944
Bash,6.817603
Bayes,6.860377
Betelgeuse,11.416373
Ceres,10.599704
Curie,6.932073
Darden,9.695201
Deimos,6.431052


In [32]:
users = users.merge(active_cohort, how='left', on='name')
users

,user_id,program,avg_daily_hits,cohort_avg_daily_hits
name,,,,
Teddy,2,Web,14.676190,9.375603
Teddy,3,Web,13.889908,9.375603
Teddy,4,Web,11.344262,9.375603
Teddy,5,Web,15.292453,9.375603
Teddy,6,Web,11.723214,9.375603
...,...,...,...,...
Darden,780,DataScience,4.333333,9.695201
Darden,781,DataScience,8.545455,9.695201
Darden,783,DataScience,6.000000,9.695201


In [33]:
users['percent_of_cohort_hits'] = users['avg_daily_hits']/users['cohort_avg_daily_hits']
not_active = users[users['percent_of_cohort_hits'] <= .50]
not_active

,user_id,program,avg_daily_hits,cohort_avg_daily_hits,percent_of_cohort_hits
name,,,,,
Teddy,13,Web,0.814433,9.375603,0.086867
Sequoia,24,Web,2.000000,6.390671,0.312956
Sequoia,41,Web,2.000000,6.390671,0.312956
Teddy,46,Web,3.000000,9.375603,0.319979
Sequoia,49,Web,2.272727,6.390671,0.355632
Ulysses,122,Web,4.185484,8.403907,0.498040
Ulysses,130,Web,2.343066,8.403907,0.278807
Voyageurs,198,Web,4.656716,9.477153,0.491362
Voyageurs,206,Web,3.795082,9.477153,0.400445


## WebDev Least Active Users
[Table Of Contents](#Table-of-Contents)

In [34]:
not_active_web = not_active[not_active['program']=='Web']['user_id']
not_active_web

name
Teddy          13
Sequoia        24
Sequoia        41
Teddy          46
Sequoia        49
Ulysses       122
Ulysses       130
Voyageurs     198
Voyageurs     206
Wrangell      232
Voyageurs     249
Voyageurs     250
Voyageurs     253
Xanadu        265
Xanadu        267
Xanadu        271
Voyageurs     278
Yosemite      300
Xanadu        315
Yosemite      321
Zion          331
Yosemite      371
Betelgeuse    409
Betelgeuse    411
Betelgeuse    425
Betelgeuse    426
Ceres         448
Deimos        497
Deimos        503
Deimos        505
Europa        523
Ceres         543
Fortuna       548
Fortuna       561
Fortuna       569
Apex          601
Ganymede      619
Ganymede      623
Ganymede      626
Ganymede      628
Hyperion      676
Bash          704
Bash          712
Bash          734
Jupiter       762
Jupiter       766
Name: user_id, dtype: int64

In [35]:
not_active_web.to_csv('Web_Stu_Inactive.csv')

## DS Least Active Users
[Table Of Contents](#Table-of-Contents)

In [36]:
not_active_ds = not_active[not_active['program']=='DataScience']['user_id']
not_active_ds


name
Bayes     470
Bayes     477
Curie     575
Curie     583
Darden    693
Darden    780
Darden    785
Name: user_id, dtype: int64

In [37]:
not_active_ds.to_csv('DS_Stu_Inactive.csv')

# Pages Accessed After Graduation
> What topics are grads continuing to reference after graduation and into their jobs (for each program)

To answer this question we will create DFs that holds all the webpages accessed by each cohort after their graduation. From there we will determine what pages are accessed the most across all cohorts. Finally, we will split the single data frame into two for, one for each program (WebDev & DS)


[Table Of Contents](#Table-of-Contents)

In [38]:
# Create an empty data frame to store the data
after_grad = pd.DataFrame()
# use zip within in a for loop to pull out the end date and cohort id combos from the cohort data frame
for cohort_id, end_date in zip(cohort.cohort_id, cohort.end_date):
    # Then for each cohort and end_date combo, only pull the rows where the class was still in session based on date
    after_grad = after_grad.append(data[(data['cohort_id'] == cohort_id) & (data['date'] >= end_date)])

after_grad

,date,time,page_viewed,user_id,cohort_id,ip,hour,day,month,name,start_date,end_date,program_id,classtype,program,location
datetime,,,,,,,,,,,,,,,,
2018-01-26 10:14:47,2018-01-26,10:14:47,/,11,1.0,97.105.19.61,10,Friday,January,Arches,2014-02-04,2014-04-22,1.0,Web-SanAntonio,Web,SanAntonio
2018-01-26 10:14:51,2018-01-26,10:14:51,javascript-i,11,1.0,97.105.19.61,10,Friday,January,Arches,2014-02-04,2014-04-22,1.0,Web-SanAntonio,Web,SanAntonio
2018-01-26 10:15:02,2018-01-26,10:15:02,javascript-i/functions,11,1.0,97.105.19.61,10,Friday,January,Arches,2014-02-04,2014-04-22,1.0,Web-SanAntonio,Web,SanAntonio
2018-01-26 11:46:29,2018-01-26,11:46:29,appendix/further-reading/javascript/hoisting.html,11,1.0,97.105.19.61,11,Friday,January,Arches,2014-02-04,2014-04-22,1.0,Web-SanAntonio,Web,SanAntonio
2018-01-26 11:46:39,2018-01-26,11:46:39,appendix,11,1.0,97.105.19.61,11,Friday,January,Arches,2014-02-04,2014-04-22,1.0,Web-SanAntonio,Web,SanAntonio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-02 08:06:48,2020-11-02,08:06:48,spring,638,57.0,24.243.32.134,8,Monday,November,Ganymede,2020-03-23,2020-08-20,2.0,Web-Dallas,Web,Dallas
2020-11-02 08:06:51,2020-11-02,08:06:51,spring/setup,638,57.0,24.243.32.134,8,Monday,November,Ganymede,2020-03-23,2020-08-20,2.0,Web-Dallas,Web,Dallas
2020-11-02 15:56:05,2020-11-02,15:56:05,spring/fundamentals/controllers,638,57.0,24.243.32.134,15,Monday,November,Ganymede,2020-03-23,2020-08-20,2.0,Web-Dallas,Web,Dallas


In [39]:
# turn the after_grad df to a df grouped program and size 
after_grad = pd.DataFrame(after_grad[['program_id', 'classtype', 'page_viewed']].groupby(['program_id', 'classtype', 'page_viewed'])['page_viewed'].size())
# rename the column to hits
after_grad.columns=['hits']
# reset the index for easier splitting
after_grad = after_grad.reset_index()
after_grad

,program_id,classtype,page_viewed,hits
0,1.0,Web-SanAntonio,/,1610
1,1.0,Web-SanAntonio,0_Classification_Algorithms,2
2,1.0,Web-SanAntonio,1-fundamentals/1.1-intro-to-data-science,10
3,1.0,Web-SanAntonio,1-fundamentals/2.4-more-excel-features,1
4,1.0,Web-SanAntonio,1-fundamentals/AI-ML-DL-timeline.jpg,10
...,...,...,...,...
2645,3.0,DataScience-SanAntonio,timeseries/working-with-time-series-data-with-...,4
2646,4.0,Web-Houston,/,1
2647,4.0,Web-Houston,content/html-css,2
2648,4.0,Web-Houston,content/html-css/gitbook/images/favicon.ico,1


In [40]:
after_grad.program_id.unique()

array([1., 2., 3., 4.])

In [41]:
#split the data frame by program_id 
program1 = after_grad[after_grad['program_id']==1]
program2 = after_grad[after_grad['program_id']==2]
program3 = after_grad[after_grad['program_id']==3]
program4 = after_grad[after_grad['program_id']==4]

## By Program
* Describe the each program to find and quartiles use best judgement to determine the threshold for accessing after graduation (3rd quartile)
* Then find the pages accessed by each program

[Table Of Contents](#Table-of-Contents)

In [42]:
program1.describe()

,program_id,hits
count,702.0,702.000000
mean,1.0,42.861823
std,0.0,100.904353
min,1.0,1.000000
25%,1.0,2.000000
50%,1.0,10.000000
75%,1.0,38.000000
max,1.0,1610.000000


In [43]:
program1[program1['hits']>=150]

,program_id,classtype,page_viewed,hits
0,1.0,Web-SanAntonio,/,1610
63,1.0,Web-SanAntonio,appendix,409
208,1.0,Web-SanAntonio,content/html-css,228
285,1.0,Web-SanAntonio,content/laravel/intro,203
475,1.0,Web-SanAntonio,html-css,542
493,1.0,Web-SanAntonio,html-css/elements,201
494,1.0,Web-SanAntonio,html-css/forms,153
495,1.0,Web-SanAntonio,html-css/introduction,157
498,1.0,Web-SanAntonio,index.html,1011
504,1.0,Web-SanAntonio,java-i,444


In [44]:
program1[program1['hits']>=150].to_csv('Program1.csv')

In [45]:
program2.describe()

,program_id,hits
count,1575.0,1575.000000
mean,2.0,75.760000
std,0.0,338.835358
min,2.0,1.000000
25%,2.0,2.000000
50%,2.0,8.000000
75%,2.0,32.000000
max,2.0,9643.000000


In [46]:
program2[program2['hits']>=32]

,program_id,classtype,page_viewed,hits
702,2.0,Web-Dallas,/,9643
712,2.0,Web-Dallas,1-fundamentals/1-fundamentals-overview,57
714,2.0,Web-Dallas,1-fundamentals/1.1-intro-to-data-science,468
715,2.0,Web-Dallas,1-fundamentals/1.2-data-science-pipeline,88
716,2.0,Web-Dallas,1-fundamentals/1.3-pipeline-demo,63
...,...,...,...,...
2268,2.0,Web-Dallas,web-design/intro,64
2269,2.0,Web-Dallas,web-design/ui/color,41
2270,2.0,Web-Dallas,web-design/ui/typography,55
2271,2.0,Web-Dallas,web-design/ui/visuals,32


In [47]:
program2[program2['hits']>=32].to_csv('Program2.csv')

In [48]:
program3.describe()

,program_id,hits
count,369.0,369.000000
mean,3.0,15.813008
std,0.0,43.308176
min,3.0,1.000000
25%,3.0,3.000000
50%,3.0,8.000000
75%,3.0,19.000000
max,3.0,746.000000


In [49]:
program3[program3['hits']>=19]

,program_id,classtype,page_viewed,hits
2277,3.0,DataScience-SanAntonio,/,746
2278,3.0,DataScience-SanAntonio,1-fundamentals/1.1-intro-to-data-science,102
2290,3.0,DataScience-SanAntonio,1-fundamentals/AI-ML-DL-timeline.jpg,103
2291,3.0,DataScience-SanAntonio,1-fundamentals/DataToAction_v2.jpg,20
2292,3.0,DataScience-SanAntonio,1-fundamentals/modern-data-scientist.jpg,104
...,...,...,...,...
2619,3.0,DataScience-SanAntonio,stats/simulation,20
2635,3.0,DataScience-SanAntonio,timeseries/acquire,30
2636,3.0,DataScience-SanAntonio,timeseries/explore,21
2637,3.0,DataScience-SanAntonio,timeseries/modeling-lesson1,23


In [50]:
program3[program3['hits']>=19].to_csv('Program3.csv')

In [51]:
program4.describe()

,program_id,hits
count,4.0,4.00
mean,4.0,1.25
std,0.0,0.50
min,4.0,1.00
25%,4.0,1.00
50%,4.0,1.00
75%,4.0,1.25
max,4.0,2.00


In [52]:
program4[program4['hits']>=1]

,program_id,classtype,page_viewed,hits
2646,4.0,Web-Houston,/,1
2647,4.0,Web-Houston,content/html-css,2
2648,4.0,Web-Houston,content/html-css/gitbook/images/favicon.ico,1
2649,4.0,Web-Houston,content/html-css/introduction.html,1


In [53]:
program4[program4['hits']>=1].to_csv('Program4.csv')

# Suspicious Activity 
>Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses? Any odd user-agents? 

To answer this question I will get determine which hours of the day are the least active and from there, which users/ips are accessing the lessons during those times to see if there is anything supicious going on. I will use the entire data set (data) for this task.

[Table Of Contents](#Table-of-Contents)

In [54]:
# Create a data frame to see the probaility for hits by hour of the day through hours.value_counts()/df.shape[0]...
# and reset the index
hour_probaility = pd.DataFrame(data['hour'].value_counts()/data.shape[0]).reset_index()
# rename the columns
hour_probaility.columns=['hour', 'probaility']
# sort the values
hour_probaility = hour_probaility.sort_values(by='hour')
hour_probaility

,hour,probaility
17,0,0.006933
19,1,0.003717
20,2,0.002210
23,3,0.001187
22,4,0.001302
21,5,0.002140
18,6,0.004741
16,7,0.012619
7,8,0.057351
0,9,0.143499


In [55]:
hour_probaility['probaility'].describe()

count    24.000000
mean      0.041667
std       0.042119
min       0.001187
25%       0.006385
50%       0.023811
75%       0.076795
max       0.143499
Name: probaility, dtype: float64

Based on the above information we can safely ignore ,for now, the highest probability of access the curriculum (9AM server time) because that is when class starts. We will want to focus in on time that is below 2% probaility

In [56]:
hours = list(hour_probaility[hour_probaility['probaility']<= .02]['hour'])
hours

[0, 1, 2, 3, 4, 5, 6, 7, 23]

Do the same thing for IP address that was done for hours

In [57]:
# Create a data frame to see the probaility for hits by IP address through ip.value_counts()/df.shape[0]...
# and reset the index
ip_probaility = pd.DataFrame(data['ip'].value_counts()/data.shape[0]).reset_index()
# rename the columns
ip_probaility.columns=['ip', 'probaility']
# sort the values
ip_probaility = ip_probaility.sort_values(by='ip')
ip_probaility

,ip,probaility
539,100.11.170.135,0.000158
239,100.19.73.35,0.000567
1722,100.35.67.162,0.000015
1823,103.137.12.164,0.000013
4134,103.208.220.130,0.000001
...,...,...
883,99.97.208.13,0.000061
1308,99.97.210.12,0.000025
325,99.98.32.165,0.000371
864,99.98.32.97,0.000064


In [58]:
ip_probaility['probaility'].describe()

count    4339.000000
mean        0.000230
std         0.006157
min         0.000001
25%         0.000004
50%         0.000010
75%         0.000038
max         0.395546
Name: probaility, dtype: float64

The IP address that has nearly a 40% probaility of hitting the curriculum is fairly suspicious given that the 3rd quartile value is .000038 and the STD is only .006157. Let's look up what this IP address is and be sure to include it in our suspicion list.  We will also include any ip address has a probability of the 1st quartile or below because we expect students to use the hit the website often.

In [59]:
ip_probaility[ip_probaility['probaility'] == ip_probaility['probaility'].max()]['ip']

0    97.105.19.58
Name: ip, dtype: object

In [60]:
# Create the list of ip addresses based on the above criteria
ips = list(ip_probaility[(ip_probaility['probaility'] == ip_probaility['probaility'].max()) | (ip_probaility['probaility'] <= .000004)]['ip']
)
# For curiousity, let's find what percentage of all unique ips this represents
len(ips)/len(list(data['ip'].unique()))

0.24775293846508412

Let's look at the IP with the highest probaility first and find the number of day's it's been active

In [61]:
# Convert the date field into a datetime obj
data['date'] = pd.to_datetime(data.date)
# Then find how many days it has been active
pd.to_datetime(data[data['ip']=='97.105.19.58']['date'].max()) - pd.to_datetime(data[data['ip']=='97.105.19.58']['date'].min())

Timedelta('616 days 00:00:00')

Total number of days of the data set?

In [62]:
pd.to_datetime(data['date'].max()) - pd.to_datetime(data['date'].min())

Timedelta('1011 days 00:00:00')

Create a DF that includes only the hours of the day and ip addresses we are interested in.

In [63]:
# slice the data df by the least probable hours list AND the ip list
possible = data[(data['hour'].isin(hours)) & (data['ip'].isin(ips))]
# print the head to test results
possible

,date,time,page_viewed,user_id,cohort_id,ip,hour,day,month,name,start_date,end_date,program_id,classtype,program,location
datetime,,,,,,,,,,,,,,,,
2018-02-04 23:09:55,2018-02-04,23:09:55,java-i/strings,52,21.0,99.203.27.154,23,Sunday,February,Sequoia,2017-09-27,2018-02-15,2.0,Web-Dallas,Web,Dallas
2018-02-04 23:10:55,2018-02-04,23:10:55,java-i/control-statements-and-loops,52,21.0,99.203.27.154,23,Sunday,February,Sequoia,2017-09-27,2018-02-15,2.0,Web-Dallas,Web,Dallas
2018-02-08 23:17:07,2018-02-08,23:17:07,/,92,19.0,173.174.178.51,23,Thursday,February,Quincy,2017-06-05,2017-09-22,1.0,Web-SanAntonio,Web,SanAntonio
2018-02-27 23:47:56,2018-02-27,23:47:56,slides/methods,52,21.0,99.203.26.234,23,Tuesday,February,Sequoia,2017-09-27,2018-02-15,2.0,Web-Dallas,Web,Dallas
2018-02-28 23:31:34,2018-02-28,23:31:34,javascript-ii/es6,4,22.0,107.77.66.47,23,Wednesday,February,Teddy,2018-01-08,2018-05-17,2.0,Web-Dallas,Web,Dallas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-10 23:07:56,2020-09-10,23:07:56,stats/hypothesis-testing-overview,696,59.0,107.77.220.12,23,Thursday,September,Darden,2020-07-13,2021-01-12,3.0,DataScience-SanAntonio,DataScience,SanAntonio
2020-10-13 05:38:14,2020-10-13,05:38:14,html-css/forms,759,62.0,172.58.99.231,5,Tuesday,October,Jupiter,2020-09-21,2021-03-30,2.0,Web-Dallas,Web,Dallas
2020-10-21 06:59:36,2020-10-21,06:59:36,html-css/css-i/flexbox/flexbox-fundamentals,759,62.0,172.58.102.252,6,Wednesday,October,Jupiter,2020-09-21,2021-03-30,2.0,Web-Dallas,Web,Dallas


Create a data frame that gets the number of hits per hour from each ip from the possible data frame then describe the data frame to see the basic stats on hits

In [64]:
# Group by the IP addresses and then resample by hour and reset the index
ip_hits_by_hour = possible.groupby('ip').resample('H').size().reset_index()
# Rename the columns
ip_hits_by_hour.columns=['ip', 'datehour', 'hits']
# sort the values by ip and then datehour
ip_hits_by_hour = ip_hits_by_hour.sort_values(by=['ip', 'datehour'])
# set date hour to the index
ip_hits_by_hour = ip_hits_by_hour.set_index('datehour')
# describe the data frame
ip_hits_by_hour.describe()

,hits
count,14812.000000
mean,0.220429
std,1.681340
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,43.000000


In [65]:
# investigate the max number of hits
ip_hits_by_hour[ip_hits_by_hour['hits'] == 43]

,ip,hits
datehour,,
2018-10-08 07:00:00,97.105.19.58,43


In [66]:
# see what pages this person accssed on that day and hour
data[(data['ip'] == '97.105.19.58') & (data['date']=='2018-10-08') & (data['hour']==7)]['page_viewed']

datetime
2018-10-08 07:11:15                                                    /
2018-10-08 07:11:17                                                  toc
2018-10-08 07:11:20                                             html-css
2018-10-08 07:11:23                                html-css/introduction
2018-10-08 07:12:06                                    html-css/elements
2018-10-08 07:12:45                                       html-css/forms
2018-10-08 07:14:50                                       html-css/css-i
2018-10-08 07:14:52                          html-css/css-i/introduction
2018-10-08 07:15:09              html-css/css-i/selectors-and-properties
2018-10-08 07:16:38                             html-css/css-i/box-model
2018-10-08 07:17:34                           html-css/css-i/positioning
2018-10-08 07:18:10                                      html-css/css-ii
2018-10-08 07:18:14                        html-css/css-ii/media-queries
2018-10-08 07:18:48                       

## Possible Web Scraper!
Due to the web pages accessed and the times between accessing each page, as well as the ip address being assoicated with the max probaility of this ip address being used, it is reasonable to infer that ***IP Address: 97.105.19.58*** address is engaged in web scraping.

[Table Of Contents](#Table-of-Contents)